<a href="https://colab.research.google.com/github/mamuncseru/deep_understanding_deep_learning/blob/main/DUDL_RNN_intro2RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## import libraries
import torch
import torch.nn as nn
import numpy as np

### Explore the RNN type

In [2]:
# set layer parameters
input_size = 9      # number of features to extract (e.g., number of data channels)
hidden_size = 16    # number of units in the hidden state
num_layers = 1      # number of vertical stacks of hidden layers (note: only the final layer)
actfun = 'tanh'
bias = True

# create an RNN instance
rnn = nn.RNN(input_size, hidden_size, num_layers, nonlinearity=actfun, bias=bias)
print(rnn)

RNN(9, 16)


In [3]:
## check out the source code
??nn.RNN

In [4]:
# set data parameters
seqlength = 5
batchsize = 2

# create some data
X = torch.rand(seqlength, batchsize, input_size)

# create a hidden layer (typically intialized as zeros)
hidden = torch.zeros(num_layers, batchsize, hidden_size)

# run some through the model and show the output sizes
y, h = rnn(X, hidden)
print(f'Input shape: {list(X.shape)}')
print(f'Hidden shape: {list(h.shape)}')
print(f'Output shape: {list(y.shape)}')

Input shape: [5, 2, 9]
Hidden shape: [1, 2, 16]
Output shape: [5, 2, 16]


In [5]:
## Default hidden state is all zeros if nothing specified
y, h1 = rnn(X, hidden)
print(h1), print('\n\n')

y, h2 = rnn(X)
print(h), print('\n\n')

# they're the same! (meaning default=zeros)
print(h1-h2)

tensor([[[-0.2087,  0.2513,  0.5269,  0.4155,  0.4718,  0.1919,  0.1799,
           0.2200, -0.0055,  0.7245,  0.1246,  0.6388,  0.1293,  0.2924,
           0.4164,  0.6553],
         [ 0.0058,  0.1659,  0.4529,  0.6000,  0.3870,  0.2117,  0.0750,
           0.0519,  0.1599,  0.6219,  0.1927,  0.6066,  0.2927,  0.2415,
           0.5760,  0.7715]]], grad_fn=<StackBackward0>)



tensor([[[-0.2087,  0.2513,  0.5269,  0.4155,  0.4718,  0.1919,  0.1799,
           0.2200, -0.0055,  0.7245,  0.1246,  0.6388,  0.1293,  0.2924,
           0.4164,  0.6553],
         [ 0.0058,  0.1659,  0.4529,  0.6000,  0.3870,  0.2117,  0.0750,
           0.0519,  0.1599,  0.6219,  0.1927,  0.6066,  0.2927,  0.2415,
           0.5760,  0.7715]]], grad_fn=<StackBackward0>)



tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]],
       grad_fn=<SubBackward0>)


In [6]:
# check out the learned parameters and their sizes
for p in rnn.named_parameters():
    if 'weight' in p[0]:
        print(f'{p[0]} has size {list(p[1].shape)}')

weight_ih_l0 has size [16, 9]
weight_hh_l0 has size [16, 16]


## Create a DL model class

In [7]:
class RNNnet(nn.Module):
    def __init__(self, input_size, num_hidden, num_layers):
        super().__init__()

        # store parameters
        self.input_size = input_size
        self.num_hidden = num_hidden
        self.num_layers = num_layers

        ## RNN Layer
        self.rnn = nn.RNN(input_size, num_hidden, num_layers)

        # linear layer for output
        self.out = nn.Linear(num_hidden, 1)

    def forward(self, x):

        print(f'Input: {list(x.shape)}')

        # initialize hidden state for first input
        hidden= torch.zeros(self.num_layers, batchsize, self.num_hidden)
        print(f'Hidden: {list(hidden.shape)}')

        # run through the RNN layer
        y, hidden = self.rnn(x, hidden)
        print(f'RNN-out: {list(y.shape)}')
        print(f'RNN-hidden: {list(hidden.shape)}')

        # pass the RNN output through the linear output layer
        o = self.out(y) 
        print(f'Output: {list(o.shape)}')

        return o, hidden



In [8]:
# create an instance of the model and inspect
net = RNNnet(input_size, hidden_size, num_layers)
print(net), print(' ')

# and checkout all learnable parameters
for p in net.named_parameters():
    print(f'{p[0]} has size {list(p[1].shape)}')

RNNnet(
  (rnn): RNN(9, 16)
  (out): Linear(in_features=16, out_features=1, bias=True)
)
 
rnn.weight_ih_l0 has size [16, 9]
rnn.weight_hh_l0 has size [16, 16]
rnn.bias_ih_l0 has size [16]
rnn.bias_hh_l0 has size [16]
out.weight has size [1, 16]
out.bias has size [1]


In [10]:
# test the model with some fake data
# create some data
X = torch.rand(seqlength, batchsize, input_size)
y = torch.rand(seqlength, batchsize, 1)
yHat, y = net(X)

# try a loss function
lossfun = nn.MSELoss()
lossfun(yHat, y)

Input: [5, 2, 9]
Hidden: [1, 2, 16]
RNN-out: [5, 2, 16]
RNN-hidden: [1, 2, 16]
Output: [5, 2, 1]


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([1, 2, 16])) that is different to the input size (torch.Size([5, 2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(0.1312, grad_fn=<MseLossBackward0>)